In [12]:
from __future__ import print_function
from cmath import nan
from os import truncate

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import col,isnan,when,count,lower,regexp_replace,udf
from pyspark.ml.feature import Imputer,Tokenizer,StopWordsRemover,HashingTF, IDF, CountVectorizer, VectorAssembler, RegexTokenizer
from pyspark.ml.classification import LogisticRegression,RandomForestClassifier,MultilayerPerceptronClassifier,LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from langdetect import detect
from textblob import TextBlob



import nltk
from nltk.corpus import stopwords
import re

In [13]:
# pip install pyspark

In [17]:
spark = SparkSession.builder.appName('hw3').getOrCreate()
data = spark.read.csv("./sentiments.csv", sep=',', multiLine=True,header = True)

In [26]:
data = data.filter((data.lang == 'en'))

In [27]:
data.show()
tweet = data

+--------------------+----+---------+
|               tweet|lang|sentiment|
+--------------------+----+---------+
|@IslandGirlPRV @B...|  en|        2|
|@chrislongview Wa...|  en|        2|
|#censorship #Hunt...|  en|        2|
|"""IS THIS WRONG?...|  en|        2|
|In 2020, #NYPost ...|  en|        2|
|►► Tell Politicia...|  en|        2|
|Proof  Bidens are...|  en|        2|
|Now Open! Create ...|  en|        2|
|#JoeBiden was the...|  en|        2|
|Y’all Just Lockin...|  en|        2|
|@tedcruz @cc125 #...|  en|        2|
|#IceCube isn’t a ...|  en|        2|
|BREAKING — Twitte...|  en|        2|
|"Comments on this...|  en|        2|
|https://t.co/khrZ...|  en|        2|
|I’m going to shar...|  en|        2|
|In an effort to f...|  en|        2|
|Twitter is doing ...|  en|        2|
|#JoeBiden calls h...|  en|        2|
|@ProjectLincoln @...|  en|        2|
+--------------------+----+---------+
only showing top 20 rows



In [28]:
data.groupBy('lang').count().show()

+----+------+
|lang| count|
+----+------+
|  en|512580|
+----+------+



In [8]:
def sentiment_analysis(tweet):
    # Determine polarity
        polarity = TextBlob(" ".join(tweet)).sentiment.polarity
    # Classify overall sentiment
        if polarity > 0:
        # positive
            sentiment = 1
        elif polarity == 0:
        # neutral
            sentiment = 2
        else:
        # negative
             sentiment = 2
        return sentiment

In [9]:
polarityUDF= udf(lambda x: sentiment_analysis(x),StringType())
    #tweet=tweet.select(polarityUDF(col("tweet")).alias("tweet") )
tweet=data.withColumn("sentiment", polarityUDF("tweet"))
    

In [10]:
# tweet = tweet.sampleBy("sentiment", fractions={"1": 1, "2": 0.01}, seed=0)

In [29]:
tweet.groupBy('sentiment').count().show()

+---------+------+
|sentiment| count|
+---------+------+
|        1|  5324|
|        2|507256|
+---------+------+



In [9]:
# import re, nltk, spacy, string

# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from pprint import pprint

# import pyLDAvis
# import pyLDAvis.sklearn
# import matplotlib.pyplot as plt
# %matplotlib inline

# from plotly.offline import plot
# import plotly.graph_objects as go
# import plotly.express as px

# import spacy

# import matplotlib as mpl
# import matplotlib.pyplot as plt
# %matplotlib inline

# from subprocess import check_output
# from wordcloud import WordCloud, STOPWORDS
# import pandas as pd
# import numpy as np

# df = pd.read_csv('quora_sample.csv')
# print('We have',len(df), 'questions in the data')


# def clean_text(text):
#     '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     text = text.lower()
#     text = re.sub(r'\[.*?\]', '', text)
#     text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub(r'\w*\d\w*', '', text)
#     return text

# df_clean = pd.DataFrame(df.question_text.apply(lambda x: clean_text(x)))

# nlp = spacy.load("en_core_web_sm")
# def lemmatizer(text):        
#     sent = []
#     doc = nlp(text)
#     for word in doc:
#         sent.append(word.lemma_)
#     return " ".join(sent)
    
# df_clean["question_lemmatize"] =  df_clean.apply(lambda x: lemmatizer(x['question_text']), axis=1)
# df_clean['question_lemmatize_clean'] = df_clean['question_lemmatize'].str.replace('-PRON-', '')



# mpl.rcParams['figure.figsize']=(12.0,12.0)  
# mpl.rcParams['font.size']=12            
# mpl.rcParams['savefig.dpi']=100             
# mpl.rcParams['figure.subplot.bottom']=.1 
# stopwords = set(STOPWORDS)

# wordcloud = WordCloud(
#                           background_color='white',
#                           stopwords=stopwords,
#                           max_words=500,
#                           max_font_size=40, 
#                           random_state=42
#                          ).generate(str(df_clean['question_lemmatize_clean']))

# print(wordcloud)
# fig = plt.figure(1)
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show();

# vectorizer = CountVectorizer(analyzer='word',       
#                              min_df=3,                       
#                              stop_words='english',             
#                              lowercase=True,                   
#                              token_pattern='[a-zA-Z0-9]{3,}',  
#                              max_features=5000,          
#                             )

# data_vectorized = vectorizer.fit_transform(df_clean['question_lemmatize_clean'])

# lda_model = LatentDirichletAllocation(n_components=20, # Number of topics
#                                       learning_method='online',
#                                       random_state=0,       
#                                       n_jobs = -1  # Use all available CPUs
#                                      )
# lda_output = lda_model.fit_transform(data_vectorized)

# pyLDAvis.enable_notebook()
# pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')


# def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
#     keywords = np.array(vectorizer.get_feature_names())
#     print(keywords)
#     topic_keywords = []
#     for topic_weights in lda_model.components_:
#         top_keyword_locs = (-topic_weights).argsort()[:n_words]
#         topic_keywords.append(keywords.take(top_keyword_locs))
#     return topic_keywords

# topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)

# df_topic_keywords = pd.DataFrame(topic_keywords)
# df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
# df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
# df_topic_keywords





In [30]:
tweet = tweet.select((lower(regexp_replace('tweet', "[^a-zA-Zd+ ]", "")).alias('tweet')),'sentiment')

In [31]:
tokenizer = Tokenizer(inputCol="tweet", outputCol="tweet1")
data1 = tokenizer.transform(tweet)
remover = StopWordsRemover(inputCol="tweet1", outputCol="tweet2")
data1=remover.transform(data1)

In [32]:
data1.show()

+--------------------+---------+--------------------+--------------------+
|               tweet|sentiment|              tweet1|              tweet2|
+--------------------+---------+--------------------+--------------------+
|islandgirlprv bra...|        2|[islandgirlprv, b...|[islandgirlprv, b...|
|chrislongview wat...|        2|[chrislongview, w...|[chrislongview, w...|
|censorship hunter...|        2|[censorship, hunt...|[censorship, hunt...|
|is this wrong cor...|        2|[is, this, wrong,...|[wrong, cory, boo...|
|in  nypost is bei...|        2|[in, , nypost, is...|[, nypost, censor...|
| tell politicians...|        2|[, tell, politici...|[, tell, politici...|
|proof  bidens are...|        2|[proof, , bidens,...|[proof, , bidens,...|
|now open create a...|        2|[now, open, creat...|[open, create, fr...|
|joebiden was the ...|        2|[joebiden, was, t...|[joebiden, point,...|
|yall just locking...|        2|[yall, just, lock...|[yall, locking, a...|
|tedcruz cc trump ...|   

In [33]:
cols_redundant = ["tweet","tweet1"]
sampled_data = data1.drop(*cols_redundant)

In [34]:
# from google.colab import drive
# drive.mount('/drive')


In [35]:
sampled_data.show()

+---------+--------------------+
|sentiment|              tweet2|
+---------+--------------------+
|        2|[islandgirlprv, b...|
|        2|[chrislongview, w...|
|        2|[censorship, hunt...|
|        2|[wrong, cory, boo...|
|        2|[, nypost, censor...|
|        2|[, tell, politici...|
|        2|[proof, , bidens,...|
|        2|[open, create, fr...|
|        2|[joebiden, point,...|
|        2|[yall, locking, a...|
|        2|[tedcruz, cc, tru...|
|        2|[icecube, isnt, s...|
|        2|[breaking, , twit...|
|        2|[comments, democr...|
|        2|[httpstcokhrzuhsr...|
|        2|[im, going, share...|
|        2|[effort, find, tr...|
|        2|[twitter, everyth...|
|        2|[joebiden, calls,...|
|        2|[projectlincoln, ...|
+---------+--------------------+
only showing top 20 rows



In [36]:
from pyspark.ml.feature import Imputer,Tokenizer,StopWordsRemover,HashingTF, IDF, CountVectorizer, VectorAssembler, RegexTokenizer
cv = CountVectorizer()
cv.setVocabSize(50000)
cv.setInputCol("tweet2")
cv.setOutputCol("features")
model = cv.fit(sampled_data)
tweet_vocab_length = len(model.vocabulary)

In [37]:
tweet_vocab_length

50000

In [38]:
sampled_data.show()

+---------+--------------------+
|sentiment|              tweet2|
+---------+--------------------+
|        2|[islandgirlprv, b...|
|        2|[chrislongview, w...|
|        2|[censorship, hunt...|
|        2|[wrong, cory, boo...|
|        2|[, nypost, censor...|
|        2|[, tell, politici...|
|        2|[proof, , bidens,...|
|        2|[open, create, fr...|
|        2|[joebiden, point,...|
|        2|[yall, locking, a...|
|        2|[tedcruz, cc, tru...|
|        2|[icecube, isnt, s...|
|        2|[breaking, , twit...|
|        2|[comments, democr...|
|        2|[httpstcokhrzuhsr...|
|        2|[im, going, share...|
|        2|[effort, find, tr...|
|        2|[twitter, everyth...|
|        2|[joebiden, calls,...|
|        2|[projectlincoln, ...|
+---------+--------------------+
only showing top 20 rows



In [39]:
cvdata=model.transform(sampled_data)


In [40]:
# hashingTF = HashingTF(inputCol="tweet2", outputCol="features", numFeatures=10)
# featurizedData = hashingTF.transform(sampled_data)

In [41]:
# featurizedData.show()

In [42]:
cvdata.show()

+---------+--------------------+--------------------+
|sentiment|              tweet2|            features|
+---------+--------------------+--------------------+
|        2|[islandgirlprv, b...|(50000,[0,1,166,1...|
|        2|[chrislongview, w...|(50000,[2,117,189...|
|        2|[censorship, hunt...|(50000,[1,100,145...|
|        2|[wrong, cory, boo...|(50000,[3,311,329...|
|        2|[, nypost, censor...|(50000,[0,2,4,13,...|
|        2|[, tell, politici...|(50000,[0,1,3,4,6...|
|        2|[proof, , bidens,...|(50000,[0,43,118,...|
|        2|[open, create, fr...|(50000,[1,3,4,5,6...|
|        2|[joebiden, point,...|(50000,[2,79,367]...|
|        2|[yall, locking, a...|(50000,[1,22,48,9...|
|        2|[tedcruz, cc, tru...|(50000,[1,3,7,12,...|
|        2|[icecube, isnt, s...|(50000,[0,2,5,13,...|
|        2|[breaking, , twit...|(50000,[0,2,56,11...|
|        2|[comments, democr...|(50000,[2,3,7,9,3...|
|        2|[httpstcokhrzuhsr...|(50000,[2,4007],[...|
|        2|[im, going, share

In [43]:
# ldaData=featurizedData.select(col("tweet"))

In [44]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
lda = LDA(k=5, seed=1, optimizer="em")
lda.setMaxIter(10)
ldamodel = lda.fit(cvdata)

22/11/09 23:56:50 ERROR Executor: Exception in task 0.0 in stage 37.0 (TID 29)
java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:194)
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:191)
	at org.apache.spark.graphx.impl.EdgePartitionBuilder.toEdgePartition(EdgePartitionBuilder.scala:43)
	at org.apache.spark.graphx.EdgeRDD$.$anonfun$fromEdges$1(EdgeRDD.scala:110)
	at org.apache.spark.graphx.EdgeRDD$.$anonfun$fromEdges$1$adapted(EdgeRDD.scala:105)
	at org.apache.spark.graphx.EdgeRDD$$$Lambda$4104/0x0000000801c94000.apply(Unknown Source)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsWithIndex$2(RDD.scala:907)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsWithIndex$2$adapted(RDD.scala:907)
	at org.apache.spark.rdd.RDD$$Lambda$3108/0x0000000801a03aa8.apply(Unknown Source)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadC

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/saikarthikdindi/study/657/sparkenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/4k/rw__m_1s6lb529bvhl2whsg80000gn/T/ipykernel_11769/4050723485.py", line 5, in <module>
    ldamodel = lda.fit(cvdata)
  File "/Users/saikarthikdindi/study/657/sparkenv/lib/python3.9/site-packages/pyspark/ml/base.py", line 205, in fit
    return self._fit(dataset)
  File "/Users/saikarthikdindi/study/657/sparkenv/lib/python3.9/site-packages/pyspark/ml/wrapper.py", line 379, in _fit
    java_model = self._fit_java(dataset)
  File "/Users/saikarthikdindi/study/657/sparkenv/lib/python3.9/site-packages/pyspark/ml/wrapper.py", line 376, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/Users/saikarthikdindi/study/657/sparkenv/lib/python3.9/site-packages/py4j/java_gateway.py", line 1321, in 

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
lda_path = "./lda10000"
lda.save(lda_path)

In [ ]:
ldamodel.describeTopics().show(truncate=False)

In [27]:
ldamodel.topicsMatrix()

DenseMatrix(50000, 5, [85024.243, 70861.7977, 46711.9056, 35228.1633, 19497.534, 13483.7421, 10383.8948, 10109.0007, ..., 0.8072, 0.8503, 0.9907, 0.8682, 0.9922, 0.7081, 0.7562, 0.8446], 0)

In [28]:
ldamodel.explainParams()

'checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)\ndocConcentration: Concentration parameter (commonly named "alpha") for the prior placed on documents\' distributions over topics ("theta"). (undefined)\nfeaturesCol: features column name. (default: features)\nk: The number of topics (clusters) to infer. Must be > 1. (default: 10, current: 5)\nkeepLastCheckpoint: (For EM optimizer) If using checkpointing, this indicates whether to keep the last checkpoint. If false, then the checkpoint will be deleted. Deleting the checkpoint can cause failures if a data partition is lost, so set this bit with care. (default: True)\nlearningDecay: Learning rate, set as anexponential decay rate. This should be between (0.5, 1.0] to guarantee asymptotic convergence. (default: 0.51)\nlearningOffset:

In [29]:
# ldamodel.logPerplexity(cvdata)

In [30]:
from pyspark.sql import functions as F
vocab = model.vocabulary
def get_words(token_list):
    return [vocab[token_id] for token_id in token_list]
udf_to_words = F.udf(get_words, StringType())

In [31]:


num_top_words = 100
topics = ldamodel.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                                                                                    

In [34]:
import pyLDAvis
import pyLDAvis.sklearn

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [40]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, cvdata, cv, mds='tsne')


AttributeError: ignored

In [57]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                       
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=10000,)

data_vectorized = vectorizer.fit_transform(sampled_data)

TypeError: ignored

In [33]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=185c307f7870406d04f26e5f650b2d1889ea2ea5eac77a944f5f495f9a923490
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=082b61c242414b5e23f7ed1de0f3f3226c90d1b8a39255bf14258911d8762659
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built pyLDAvis sklearn


In [42]:
transformed = ldamodel.transform(cvdata)

In [43]:
transformed.show()

+--------------------+--------------------+--------------------+
|              tweet2|            features|   topicDistribution|
+--------------------+--------------------+--------------------+
|[islandgirlprv, b...|(50000,[0,1,166,1...|[0.19910525227805...|
|[chrislongview, w...|(50000,[2,117,189...|[0.19953992798156...|
|[censorship, hunt...|(50000,[1,100,145...|[0.19893345509430...|
|[wrong, cory, boo...|(50000,[3,311,329...|[0.19993534917011...|
|[, nypost, censor...|(50000,[0,2,4,13,...|[0.20154453969945...|
|[, tell, politici...|(50000,[0,1,3,4,6...|[0.20051541328487...|
|[proof, , bidens,...|(50000,[0,43,118,...|[0.20008301690711...|
|[open, create, fr...|(50000,[1,3,4,5,6...|[0.20000870182708...|
|[joebiden, point,...|(50000,[2,79,367]...|[0.20005476513305...|
|[yall, locking, a...|(50000,[1,22,48,9...|[0.20173514446660...|
|[tedcruz, cc, tru...|(50000,[1,3,7,12,...|[0.19993925741896...|
|[icecube, isnt, s...|(50000,[0,2,5,13,...|[0.20016410986426...|
|[breaking, , twit...|(50

In [50]:
model.save('./LDAModel')